https://github.com/BAMresearch/NFDI4IngScientificWorkflowRequirements

In [1]:
!conda env list


# conda environments:
#
base                   /srv/conda
notebook             * /srv/conda/envs/notebook
postprocessing         /srv/conda/envs/postprocessing
preprocessing          /srv/conda/envs/preprocessing
processing             /srv/conda/envs/processing



# Preprocessing 

In [2]:
from nfdi_ing_workflow import (
    generate_mesh as _generate_mesh, 
    convert_to_xdmf as _convert_to_xdmf,
    poisson as _poisson,
    plot_over_line as _plot_over_line,
    substitute_macros as _substitute_macros,
    compile_paper as _compile_paper,
)

In [3]:
from pyiron_base import Project, job

In [4]:
generate_mesh = job(_generate_mesh)
convert_to_xdmf = job(_convert_to_xdmf, output_key_lst=["xdmf_file", "h5_file"])
poisson = job(_poisson, output_key_lst=["numdofs", "pvd_file", "vtu_file"])
plot_over_line = job(_plot_over_line)
substitute_macros = job(_substitute_macros)
compile_paper = job(_compile_paper)

In [5]:
pr = Project("test")
pr.remove_jobs(recursive=True, silently=True)

  0%|          | 0/6 [00:00<?, ?it/s]

In [6]:
domain_size = 2.0

## generate mesh

In [7]:
gmsh_output_file = generate_mesh(
    domain_size=domain_size,
    pyiron_project=pr
)

## convert to xdmf

In [8]:
meshio_output_dict = convert_to_xdmf(
    gmsh_output_file=gmsh_output_file,
    pyiron_project=pr,
)

# Processing

## poisson

In [9]:
poisson_dict = poisson(
    meshio_output_xdmf=meshio_output_dict.output.xdmf_file, 
    meshio_output_h5=meshio_output_dict.output.h5_file,
    pyiron_project=pr,
)

# Postprocessing

## plot over line

In [10]:
pvbatch_output_file = plot_over_line(
    poisson_output_pvd_file=poisson_dict.output.pvd_file, 
    poisson_output_vtu_file=poisson_dict.output.vtu_file,
    pyiron_project=pr,
)

## substitute macros

In [11]:
macros_tex_file = substitute_macros( 
    pvbatch_output_file=pvbatch_output_file, 
    ndofs=poisson_dict.output.numdofs, 
    domain_size=domain_size,
    pyiron_project=pr,
)

## compile paper

In [12]:
paper_output = compile_paper(
    macros_tex=macros_tex_file, 
    plot_file=pvbatch_output_file,
    pyiron_project=pr,
)

# Convert 

In [13]:
from python_workflow_definition.pyiron_base import write_workflow_json

In [14]:
workflow_json_filename = "pyiron_base_nfdi.json"

In [15]:
write_workflow_json(delayed_object=paper_output, file_name=workflow_json_filename)

# Load Workflow with aiida

In [16]:
from aiida import load_profile

load_profile()

Profile<uuid='8bb81b57acda4bf6bbc69d5eb5ae123e' name='pwd'>

In [17]:
from python_workflow_definition.aiida import load_workflow_json

In [18]:
wg = load_workflow_json(file_name=workflow_json_filename)
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

In [19]:
wg.run()

04/02/2025 02:42:49 PM <3366> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [57|WorkGraphEngine|continue_workgraph]: tasks ready to run: generate_mesh6
04/02/2025 02:42:52 PM <3366> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [57|WorkGraphEngine|update_task_state]: Task: generate_mesh6, type: PyFunction, finished.
04/02/2025 02:42:52 PM <3366> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [57|WorkGraphEngine|continue_workgraph]: tasks ready to run: convert_to_xdmf5
04/02/2025 02:42:54 PM <3366> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [57|WorkGraphEngine|update_task_state]: Task: convert_to_xdmf5, type: PyFunction, finished.
04/02/2025 02:42:54 PM <3366> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [57|WorkGraphEngine|continue_workgraph]: tasks ready to run: poisson4
04/02/2025 02:42:58 PM <3366> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [57|WorkGrap

# Load Workflow with pyiron_base

In [20]:
from python_workflow_definition.jobflow import load_workflow_json

/srv/conda/envs/notebook/lib/python3.12/site-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
/srv/conda/envs/notebook/lib/python3.12/site-packages/paramiko/transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,


In [21]:
from jobflow.managers.local import run_locally

In [22]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [23]:
result = run_locally(flow)
result

2025-04-02 14:43:07,042 INFO Started executing jobs locally
2025-04-02 14:43:07,362 INFO Starting job - generate_mesh (43b2c093-4d23-4dff-9faa-91515c28b29b)
2025-04-02 14:43:08,924 INFO Finished job - generate_mesh (43b2c093-4d23-4dff-9faa-91515c28b29b)
2025-04-02 14:43:08,925 INFO Starting job - convert_to_xdmf (54030101-696f-4d65-939f-fc0d1caa8503)
2025-04-02 14:43:10,661 INFO Finished job - convert_to_xdmf (54030101-696f-4d65-939f-fc0d1caa8503)
2025-04-02 14:43:10,663 INFO Starting job - poisson (5a5d4720-da41-44c9-a3f0-52995c086dee)
2025-04-02 14:43:13,799 INFO Finished job - poisson (5a5d4720-da41-44c9-a3f0-52995c086dee)
2025-04-02 14:43:13,801 INFO Starting job - plot_over_line (ef4ff511-cac6-4f0f-a0f6-36e049bbf9fe)
2025-04-02 14:43:15,798 INFO Finished job - plot_over_line (ef4ff511-cac6-4f0f-a0f6-36e049bbf9fe)
2025-04-02 14:43:15,799 INFO Starting job - substitute_macros (2a2432f2-612b-4598-a4a9-1ec1eeb194bf)
2025-04-02 14:43:17,144 INFO Finished job - substitute_macros (2a2432

{'43b2c093-4d23-4dff-9faa-91515c28b29b': {1: Response(output='/home/jovyan/preprocessing/square.msh', detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))},
 '54030101-696f-4d65-939f-fc0d1caa8503': {1: Response(output={'xdmf_file': '/home/jovyan/preprocessing/square.xdmf', 'h5_file': '/home/jovyan/preprocessing/square.h5'}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))},
 '5a5d4720-da41-44c9-a3f0-52995c086dee': {1: Response(output={'numdofs': 357, 'pvd_file': '/home/jovyan/processing/poisson.pvd', 'vtu_file': '/home/jovyan/processing/poisson000000.vtu'}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))},
 'ef4ff511-cac6-4f0f-a0f6-36e049bbf9fe': {1: Response(output='/home/jovyan/postprocessing/plotoverline.csv', detour=None, additi